In [ ]:
# import os
# import shutil

# data_dir = "data/"
# dist_path = "output"

# image_list = os.listdir(data_dir)
# timestamps = list()
# interval = 3
# counter = 1
# intervals = list()
# cam_number = 24

# def create_directories_with_intervals(base_dir, interval, num_folders):
#     if not os.path.exists(base_dir):
#         os.makedirs(base_dir)

#     for i in range(num_folders):
#         start = i * interval + 1
#         end = (i + 1) * interval
#         folder_name = f"{start}-{end}"  # 1-20, 21-40, ...
#         folder_path = os.path.join(base_dir, folder_name)
#         images_folder_path = os.path.join(folder_path, 'images')
        
#         os.makedirs(images_folder_path, exist_ok=True)

#         print(f"Created {images_folder_path}")

# def split_list(input_list, interval):
#     return [input_list[i:i + interval] for i in range(0, len(input_list), interval)]

# for item in image_list:
#     timestamps.append(item.split("_")[1])

# intervals = split_list(timestamps, 5)
# for interval in intervals:
#     # interval = [2984770, 2984771, 2984772] ..
#     for cam_id in interval:
#         filtered_images = [image for image in image_list if cam_id in image.split('_')[1]]
#         print(f"{cam_id} -- {len(filtered_images)}")
#         print(filtered_images)
#     print("interval end")

In [ ]:
import os
import shutil

def create_directory_structure(base_dir, interval, num_folders):
    for i in range(num_folders):
        start = i * interval + 1
        end = (i + 1) * interval
        folder_name = f"{start}-{end}"  # 1-20, 21-40, ...
        folder_path = os.path.join(base_dir, folder_name)
        
        for cam_id in range(24):
            cam_folder_name = f"cam{cam_id:02d}"
            cam_folder_path = os.path.join(folder_path, cam_folder_name, 'images')
            os.makedirs(cam_folder_path, exist_ok=True)
        print("fname ", folder_name)
        dummy(folder_path, interval, folder_name)

def dummy(folder_path, interval, folder_name):
    counter = 0
    for image in image_list:
        parts = image.split('_')
        timestamp = int(parts[1])
        camera_id = int(parts[2].split('.')[0])

        cam_folder = f"cam{camera_id:02d}"

        dest_folder = os.path.join(base_dir, folder_name, cam_folder, 'images')

def organize_images(image_list, base_dir, interval):
    for image in image_list:
        parts = image.split('_')
        timestamp = int(parts[1])
        camera_id = int(parts[2].split('.')[0])
        
        # Determine the appropriate interval folder
        interval_index = (timestamp - 1) // interval
        start = interval_index * interval + 1
        end = (interval_index + 1) * interval
        interval_folder = f"{start}-{end}"
        print(interval_folder)
        
        # Determine the appropriate camera folder
        cam_folder = f"cam{camera_id:02d}"
        
        # Construct the full destination path
        dest_folder = os.path.join(base_dir, interval_folder, cam_folder, 'images')
        dest_path = os.path.join(dest_folder, os.path.basename(image))
        print(os.path.basename(image))
        
        # Ensure the destination directory exists
        os.makedirs(dest_folder, exist_ok=True)
        
        # Copy the image to the destination path
        # shutil.copy(os.path.join(data_dir, image), dest_path)
        
        # print(f"Copied {image} to {dest_path}")

# Örnek kullanım

data_dir = "data/"
base_dir = "output"

image_list = os.listdir(data_dir)
timestamps = list()

interval = 20
num_folders = 24  # Örnek olarak 24 klasör (her biri interval genişliğinde)

# Elinizdeki resim dosyalarının listesi
image_list = os.listdir(data_dir)

# Öncelikle klasör yapısını oluştur
create_directory_structure(base_dir, interval, num_folders)

# Ardından resimleri organize et
# organize_images(image_list, base_dir, interval)

## Worked Code Below

In [1]:
import itertools
import os
import shutil

data_dir = "/mnt/Elements2/15-07-2024_Basler_DATA/test_images_oguz_2/"
base_dir = "output"
interval = 20
cam_num = 24

def copy_first_frame(source_dir):
    # Hedef dizini oluştur
    print("First Frames Copying for COLMAP..")
    target_dir = os.path.join(source_dir, "colmap_input")
    os.makedirs(target_dir, exist_ok=True)
    
    # Kaynak dizinindeki ilk tiff dosyasını bul
    for index, cam_folder in enumerate(os.listdir(os.path.join(source_dir, 'original_images'))):
        first_tiff_file = os.path.join(source_dir, 'original_images', cam_folder, 'images', f"0000.tiff")
        target_img = os.path.join(target_dir, f"{index:04d}.tiff")

        # Eğer dosya varsa, first_frames dizinine kopyala
        if os.path.exists(first_tiff_file):
            shutil.copy(first_tiff_file, target_img)
        else:
            print(f"0000.tiff dosyası bulunamadı: {first_tiff_file}")

def rename_images_in_directory(base_dir):
    print("Renaming..")
    cam_dirs = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]

    for cam_dir in cam_dirs:
        images_dir = os.path.join(base_dir, cam_dir, 'images')
        
        if os.path.exists(images_dir):
            images = sorted(os.listdir(images_dir))  # Dosyaları sıralı olarak al
            
            for i, image in enumerate(images):
                old_image_path = os.path.join(images_dir, image)
                new_image_name = f"{i:04d}.tiff"
                new_image_path = os.path.join(images_dir, new_image_name)
                
                os.rename(old_image_path, new_image_path)

def create_directory_structure(base_dir, interval, cam_num):
    start_interval_list = list()

    for i in range(cam_num):
        start = i * interval + 1
        end = (i + 1) * interval
        folder_name = f"{start}-{end}"  # 1-20, 21-40, ...
        folder_path = os.path.join(base_dir, folder_name)
        
        for cam_id in range(cam_num):
            cam_folder_name = f"cam{cam_id:02d}"
            cam_folder_path = os.path.join(folder_path, 'original_images', cam_folder_name, 'images')
            os.makedirs(cam_folder_path, exist_ok=True)
        # print("fname ", cam_folder_path)
        start_interval_list.append(f"{start}-{end}")
    return start_interval_list

start_intervals = create_directory_structure(base_dir, interval, cam_num)
start_intervals = itertools.chain(start_intervals)

image_list = os.listdir(data_dir)

timestamps = list()
for item in image_list:
    timestamps.append(item.split("_")[1])

timestamps = sorted(set(timestamps)) # get unique timestamps
output_list = list()

for ts in timestamps:
    dummy = list()
    dummy = [image for image in image_list if ts in image]
    output_list.append(dummy)

counter = 0
output_interval = next(start_intervals)

for index, sequence in enumerate(output_list):
    if counter == interval:
        rename_dir = os.path.join(base_dir, output_interval, 'original_images')
        rename_images_in_directory(rename_dir)
        copy_first_frame(os.path.join(base_dir, output_interval))
        output_interval = next(start_intervals)
        counter = 0
        
    for image in sequence:
        parts = image.split('_')
        camera_id = int(parts[2].split('.')[0])
        
        source_image_path = os.path.join(data_dir, image)
        dst_image_path = os.path.join(os.path.join(os.path.join(base_dir, output_interval)), f"original_images/cam{camera_id:02d}/images/{image}")
        
        shutil.copy(source_image_path, dst_image_path)
    
    print(f"Working on {output_interval} interval")
    counter += 1

Working on 1-20 interval
Working on 1-20 interval
Working on 1-20 interval
Working on 1-20 interval
Working on 1-20 interval
Working on 1-20 interval
Working on 1-20 interval
Working on 1-20 interval
Working on 1-20 interval
Working on 1-20 interval
Working on 1-20 interval
Working on 1-20 interval
Working on 1-20 interval
Working on 1-20 interval
Working on 1-20 interval
Working on 1-20 interval
Working on 1-20 interval
Working on 1-20 interval
Working on 1-20 interval
Working on 1-20 interval
Renaming..
First Frames Copying for COLMAP..
Working on 21-40 interval
Working on 21-40 interval
Working on 21-40 interval
Working on 21-40 interval
Working on 21-40 interval
Working on 21-40 interval
Working on 21-40 interval
Working on 21-40 interval
Working on 21-40 interval
Working on 21-40 interval
Working on 21-40 interval
Working on 21-40 interval
Working on 21-40 interval
Working on 21-40 interval
Working on 21-40 interval
Working on 21-40 interval
Working on 21-40 interval
Working on 21

## TIFF to PNG Lossless Convertion

In [31]:
import cv2

source_dir = "/home/alper/Spaceport/basler_camera_process/output/101-120/original_images/cam23/images"

for image in os.listdir(source_dir):
    img = cv2.imread(os.path.join(source_dir, image))
    img_name = image.split(".")[0]

    cv2.imwrite(f"{source_dir}/{img_name}.png", img, [cv2.IMWRITE_PNG_COMPRESSION, 0])

    os.remove(os.path.join(source_dir, image))



In [1]:
## PIXEL DEPTH TEST
import cv2

img_path = "/home/alper/Spaceport/basler_camera_process/0000.png"

img = cv2.imread(img_path)
print(img.dtype)

uint8
